### 데이터 불러오기

필요한 라이브러리와 모듈을 임포트하고 제공받은 데이터 파일은 경로로 불러와 변수에 저장해줍니다.

In [1]:
import glob  # glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환ㅠ
import os, re # os: 운영체제를 제어 # re: 정규 표현식 지원
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras


# 파일을 불러와 txt_file_path에 저장한다

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'
txt_list = glob.glob(txt_file_path)

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담는다.
raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:   # 일기 모드로 파일 열기
        raw = f.read().splitlines()   # 라인 단위로 끊어서 리스트로 읽기
        raw_corpus.extend(raw)        # 읽은 애들 하나씩 raw_corpus추가하기

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


 ### 데이터 정제하기
 
정규표현식을 사용하여 우리에게 필요한 데이터만 정제하였습니다.



In [2]:
def preprocess_sentence(sentence):
    # re는 정규표현식을 지원하는 모듈을 의미함
    sentence = sentence.lower().strip() # 소문자 변환, 양쪽 공백 지우기
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 특수문자 양쪽에 공백을 넣고
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개의 공백은 하나의 공백으로 바꿉니다
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
    sentence = sentence.strip() 
    sentence = '<start> ' + sentence + ' <end>' # 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
    return sentence

In [3]:
# 여기에 정제된 문장이 들어간다
corpus = []

for sentence in raw_corpus:           # raw_corpus에 저장한 리스트가  sentence 안에 있는동안
    if len(sentence) == 0: continue  # 문장이 공백인건 건너뛰세요
    if sentence[-1] == ":": continue # 문장 끝이 ":"인건 건너뛰세요
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

### 토큰화 하기

컴퓨터는 단어를 그대로 못 읽기 때문에 컴퓨터가 이해할 수 있는 언어인 숫자로 바꾸기

In [4]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용한다
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, # 단어장의 크기는 12,000 이상 으로 설정
        filters=" ",     # 문장을 정제했으르모 filters 설정 안해줌
        oov_token="<unk>") #12000 단어에 포함되지 못한 단어는 "<unk>"로 바꿔준다
    
    tokenizer.fit_on_texts(corpus)    # corpus를 이용해 tokenizer 내부의 단어장을 완성한다
    
    tensor = tokenizer.texts_to_sequences(corpus)  # tokenizer를 이용해 corpus를 Tensor로 변환한다
    
    # 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외한다
    tensor = [i for i in tensor if len(i) <= 15] 

    # 입력 데이터의 문장 길이를 일정하게 맞춰준다
    # 만약 문장 길이가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰준다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용한다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding="post")
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus) # tokenize() 함수로 데이터를 Tensor로 변환

[[   2   50    5 ...    0    0    0]
 [   2   17 2639 ...    0    0    0]
 [   2   36    7 ...   43    3    0]
 ...
 [   2    5  107 ...    0    0    0]
 [   2  261  200 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f93b0314400>


In [5]:
# 토큰화가 잘 되었는지 확인해보자

for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 20: break  # 여러개 확인해보고 싶어서 20개로 지정해보았따
        
# 잘 된걸 확인할 수 있다.

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to
11 : it
12 : me
13 : my
14 : in
15 : t
16 : s
17 : that
18 : on
19 : of
20 : .


### 평가 데이터셋 분리 

In [6]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2   50    5   91  297   65   57    9  969 6042    3    0    0    0]
[  50    5   91  297   65   57    9  969 6042    3    0    0    0    0]


In [7]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2,
                                                          shuffle=True, 
                                                          random_state=42)

In [8]:
print('Source Train: ', enc_train.shape)
print('Target Train: ', dec_train.shape)

Source Train:  (124810, 14)
Target Train:  (124810, 14)


### 모델 생성

In [9]:
class enerate_text(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(enerate_text, self).__init__()
        
        #입력된 텐서에는 단어사전의 인덱스가 들어있는데, 이 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔준다.
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        

        return out
    
embedding_size = 512
hidden_size = 2048
lyricist = enerate_text(tokenizer.num_words + 1, embedding_size , hidden_size)



### 모델 학습 시키기

In [10]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 512
steps_per_epoch = len(src_input) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))  # 준비한 데이터 소스로부터 데이터셋을 만듭니다
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [11]:
# 훈련에 사용할 옵티마이저(optimizer)와 손실 함수를 선택합니다:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

lyricist.compile(loss=loss, optimizer=optimizer)
lyricist.fit(dataset, epochs=10, validation_data=(enc_val, dec_val))


Epoch 1/10
609/609 [==============================] - 320s 491ms/step - loss: 3.2927 - val_loss: 2.8937
Epoch 2/10
609/609 [==============================] - 317s 520ms/step - loss: 2.7776 - val_loss: 2.5809
Epoch 3/10
609/609 [==============================] - 318s 522ms/step - loss: 2.5155 - val_loss: 2.3118
Epoch 4/10
609/609 [==============================] - 318s 523ms/step - loss: 2.2733 - val_loss: 2.0627
Epoch 5/10
609/609 [==============================] - 318s 523ms/step - loss: 2.0449 - val_loss: 1.8325
Epoch 6/10
609/609 [==============================] - 318s 522ms/step - loss: 1.8309 - val_loss: 1.6232
Epoch 7/10
609/609 [==============================] - 318s 523ms/step - loss: 1.6333 - val_loss: 1.4374
Epoch 8/10
609/609 [==============================] - 323s 531ms/step - loss: 1.4572 - val_loss: 1.2802
Epoch 9/10
609/609 [==============================] - 326s 534ms/step - loss: 1.3081 - val_loss: 1.1554
Epoch 10/10
609/609 [==============================] - 326s 535m

### 평가하기

In [35]:
def generate_text(lyricist, tokenizer, init_sentence="<start> ", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    
    while True:
        # 1. 입력받은 문장의 텐서를 입력합니다
        predict = lyricist(test_tensor) 
        
        # 2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        
        # 3. 2에서 예측된 word index를 문장 뒤에 붙입니다
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        
        if enerate_text.numpy()[0] == end_token: break  # 4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
        if test_tensor.shape[1] >= max_len: break
 

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [36]:
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)
print(generate_text)

AttributeError: 'function' object has no attribute 'numpy'

회고

점점 어려워진다. 이번에 어려웠던건 단어 길이 제한하는 거랑 val loss 구하는 거였다.
이유는,, 초반에 이걸로 엄청 해맸기 때문이다.
팀원님들의 도움으로 for문과 if 문으로 잘 해결할 수 있었다! 분명히 어떻게 하는지 아는데
막상 적용하면 왜이렇게 오류가 날까?
그리고 틀려도 도대체 어디서 뭐가 틀린건지 도저히 모르겠는 것도 많았다.

맨 마지막거는 ,,
검색을 해보고 고치기도 하고 책도 찾아보고 아,, 
뭐가 틀렸길래 대체 가사가 안나올까 정말 모르겠다.
ㅜ.ㅜ

참고

    
https://www.tensorflow.org/tutorials/keras/overfit_and_underfit?hl=ko#%ED%81%B0_%EB%AA%A8%EB%8D%B8_%EB%A7%8C%EB%93%A4%EA%B8%B0
    
https://github.com/nameunji/aiffel_project/blob/main/lyricist/lyricist.py

https://daje0601.tistory.com/196?category=952899

https://velog.io/@p2yeong/Over-fitting-vs.-Under-fitting

다른데서 본것도 있는데 하다가 지워버려서 찾을수가 없게 되었다 ㅠㅠ.